Provider Types and highest revenue procedures (non-generic or new patient office visits)
Look at both per procedure average and/or total revenue from procedure

-Drop office visits from description column, and hospital  
-pivot table to group (first by provider type then by procedure)  
-save pivot table as dataframe and sort  
-sophia did something with a for loop that i dont understand  
-combine similar type of procedure using something that just looks at stuff before the comma and use a function and loop to say "if things before the comma are identical, group these things together"

EDA  
value_counts  
pivot_table: provider type and description/codes; NPI and description/codes  
bar chart  
histogram  
check for null values  
info  

In [33]:
import pandas as pd

In [34]:
hcpcs = pd.read_pickle("../data/hcpcs_pay.pkl")

In [35]:
hcpcs['Entity Type of the Provider'].unique()

array(['I', 'O'], dtype=object)

In [36]:
hcpcs['Provider Type'].nunique()

94

In [37]:
hcpcs['Provider Type'].unique()

array(['Internal Medicine', 'Pathology', 'Anesthesiology',
       'Family Practice', 'Obstetrics & Gynecology', 'General Surgery',
       'Urology', 'Cardiac Surgery', 'Physician Assistant',
       'Physical Therapist in Private Practice', 'Dermatology',
       'Licensed Clinical Social Worker', 'Optometry',
       'Mass Immunizer Roster Biller',
       'Physical Medicine and Rehabilitation', 'Radiation Oncology',
       'Infectious Disease', 'Orthopedic Surgery',
       'Certified Clinical Nurse Specialist', 'Endocrinology',
       'Chiropractic', 'Psychiatry', 'Diagnostic Radiology',
       'Centralized Flu', 'Rheumatology', 'General Practice', 'Neurology',
       'Emergency Medicine', 'Nephrology', 'Ophthalmology',
       'Nurse Practitioner', 'Pulmonary Disease', 'Otolaryngology',
       'Plastic and Reconstructive Surgery', 'Psychologist, Clinical',
       'Cardiology', 'Ambulance Service Provider',
       'Interventional Cardiology', 'Gastroenterology', 'Hospitalist',
       'Aud

In [38]:
hcpcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9847443 entries, 0 to 9847442
Data columns (total 23 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Credentials of the Provider                               object 
 4   Entity Type of the Provider                               object 
 5   Street Address 1 of the Provider                          object 
 6   City of the Provider                                      object 
 7   Zip Code of the Provider                                  object 
 8   State Code of the Provider                                object 
 9   Country Code of the Provider                              object 
 10  Provider Type                 

In [39]:
hcpcs['Medicare Participation Indicator'].unique()

array(['Y', 'N'], dtype=object)

In [40]:
hcpcs['HCPCS Drug Indicator'].unique()

array(['N', 'Y'], dtype=object)

In [41]:
hcpcs['Number of Services'].unique()

array([  100. ,    26. ,    52. , ..., 23387.4,  2522.7,   176.2])

In [42]:
baby_hcpcs = hcpcs[['National Provider Identifier', 
                    'Entity Type of the Provider', 
                    'Provider Type', 
                    'Medicare Participation Indicator', 
                    'HCPCS Code', 
                    'HCPCS Description', 
                    'Number of Services', 
                    'Average Medicare Allowed Amount', 
                    'Average Submitted Charge Amount', 
                    'Average Medicare Payment Amount', 
                    'Average Medicare Standardized Amount']]

In [43]:
baby_hcpcs.head()

,National Provider Identifier,Entity Type of the Provider,Provider Type,Medicare Participation Indicator,HCPCS Code,HCPCS Description,Number of Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,1003000126,I,Internal Medicine,Y,99217,Hospital observation care discharge,100.0,73.398800,325.780000,56.827200,57.492400
1,1003000126,I,Internal Medicine,Y,99218,Hospital observation care typically 30 minutes,26.0,100.080000,449.000000,78.460000,79.306154
2,1003000126,I,Internal Medicine,Y,99219,Hospital observation care typically 50 minutes,52.0,136.380000,614.000000,102.807692,103.895385
3,1003000126,I,Internal Medicine,Y,99220,Hospital observation care typically 70 minutes...,59.0,190.363729,755.932203,141.293559,142.865763
4,1003000126,I,Internal Medicine,Y,99221,"Initial hospital inpatient care, typically 30 ...",16.0,101.680000,462.812500,79.710000,80.750000


In [44]:
baby_hcpcs.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9847443 entries, 0 to 9847442
Data columns (total 11 columns):
 #   Column                                Non-Null Count    Dtype  
---  ------                                --------------    -----  
 0   National Provider Identifier          9847443 non-null  int64  
 1   Entity Type of the Provider           9847443 non-null  object 
 2   Provider Type                         9847443 non-null  object 
 3   Medicare Participation Indicator      9847443 non-null  object 
 4   HCPCS Code                            9847443 non-null  object 
 5   HCPCS Description                     9847443 non-null  object 
 6   Number of Services                    9847443 non-null  float64
 7   Average Medicare Allowed Amount       9847443 non-null  float64
 8   Average Submitted Charge Amount       9847443 non-null  float64
 9   Average Medicare Payment Amount       9847443 non-null  float64
 10  Average Medicare Standardized Amount  9847443 non-null

In [45]:
baby_hcpcs['HCPCS Description'].nunique()

5467

In [46]:
baby_hcpcs.loc[baby_hcpcs['HCPCS Description'].str.contains('office')]['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes                                                                                                                                                                                          457843
Established patient office or other outpatient, visit typically 25 minutes                                                                                                                                                                                          428268
New patient office or other outpatient visit, typically 45 minutes                                                                                                                                                                                                  179569
New patient office or other outpatient visit, typically 30 minutes                                                                                                                                     

In [47]:
baby_hcpcs = baby_hcpcs[~baby_hcpcs['HCPCS Description'].str.contains('patient office')]

In [48]:
baby_hcpcs.loc[baby_hcpcs['HCPCS Description'].str.contains('patient office')]['HCPCS Description'].value_counts()

Series([], Name: HCPCS Description, dtype: int64)

In [49]:
baby_hcpcs.shape

(8204794, 11)

In [50]:
baby_hcpcs['monetary_total'] = baby_hcpcs['Average Medicare Allowed Amount']*baby_hcpcs['Number of Services']
baby_hcpcs.head()

,National Provider Identifier,Entity Type of the Provider,Provider Type,Medicare Participation Indicator,HCPCS Code,HCPCS Description,Number of Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,monetary_total
0,1003000126,I,Internal Medicine,Y,99217,Hospital observation care discharge,100.0,73.398800,325.780000,56.827200,57.492400,7339.88
1,1003000126,I,Internal Medicine,Y,99218,Hospital observation care typically 30 minutes,26.0,100.080000,449.000000,78.460000,79.306154,2602.08
2,1003000126,I,Internal Medicine,Y,99219,Hospital observation care typically 50 minutes,52.0,136.380000,614.000000,102.807692,103.895385,7091.76
3,1003000126,I,Internal Medicine,Y,99220,Hospital observation care typically 70 minutes...,59.0,190.363729,755.932203,141.293559,142.865763,11231.46
4,1003000126,I,Internal Medicine,Y,99221,"Initial hospital inpatient care, typically 30 ...",16.0,101.680000,462.812500,79.710000,80.750000,1626.88


In theory, I now have a dataframe with several unnecessary columns dropped and any HCPCS Descriptions that included "patient office" are now gone as well. I have added a money total column for use in the pivot tables. Next step is to sort by monetary total and then create a pivot table that groups first by provider type then by procedure.

In [62]:
total_by_provider = baby_hcpcs.groupby("Provider Type")["monetary_total"]
total_by_provider.head(10)

0           7339.88
1           2602.08
2           7091.76
3          11231.46
4           1626.88
             ...   
4054194     4546.30
4054195     6901.48
4054196    12754.80
4054197     6536.40
5202759     1527.72
Name: monetary_total, Length: 932, dtype: float64

In [52]:
#baby_hcpcs_pivot_prov['total']=baby_hcpcs_pivot_prov['Average Medicare Allowed Amount']*baby_hcpcs_pivot_prov['Number of Services']

In [53]:
#baby_hcpcs_pivot_prov.head()

In [54]:
#baby_hcpcs_pivot_prov.shape

In [55]:
#baby_hcpcs_pivot_proc = baby_hcpcs.pivot_table(index=['HCPCS Code', 'HCPCS Description'], values=['Average Medicare Allowed Amount', 'Number of Services'], aggfunc='sum')
#baby_hcpcs_pivot_proc.head()

In [56]:
#baby_hcpcs_pivot_proc['total']=baby_hcpcs_pivot_proc['Average Medicare Allowed Amount']*baby_hcpcs_pivot_proc['Number of Services']
#baby_hcpcs_pivot_proc.head()